

#Author: Srikaran Elakurthy

# Description
Import the processed cleaned over sampled lyrics dataset and divide inro train set and test set and store them into csv files for further use of them in Naivebayes model.

Implementing the logistic multi class classification on the above dataset.
Performing Cross validation with kfolds =5 and extract average accuracy.

Predict the test data and store the results.

Extracting the ngrams of only rock data set and ngrams of all classes dataset except the rock .
Then we intersect the ngrams  of both and find the common ngrams and inturn we again find common ngrams with the  ngrams of whole train dataset and ngrams which we obtained from above intersection .

The resulted ngrams rock weights will be extracted from the model and decrease by a factor and predict the test set with the modified model and store the results.

*Detailed description is written for many parts of the code below. Please read through for the same.

#Inputs and Outputs

Inputs:
> cleaneddata_oversampled.csv - The oversampled data distribution associated with the increase in the records of other genres and decrease in the records of the Rock genre is provided as an input


Outputs:
> oversampling_train.csv - The train set on which the model was trained.
> oversampling_test.csv - The test set on which the model was trained.
> logioversamp_results.txt - The classification report of the trained model over the test set.
> finalreslogiovrsam.csv - The actual and predicted values of the trained classifier on the test set.
> finalreslogiovrsam_rockreduced.csv - The actual and predcited values of the weight adjusted model on the rock genre.
> finallogioversamp_intersect_results_rockreduced10.txt - The classification metrics for the reduced model weights for the Rock genre by 10 percent.
> finallogioversamp_intersect_results_rockreduced10.csv - Actual and predicted values of the reduced model weights for the Rock genre by 10 percent.

> finallogiovrsam_reducerock.pkl - The weight adjusted model written into a file.
> Final_models/logiovrsam.pkl - Storing the model trained on the overasmpled dataset.
> finallogioversamp_results_rockreduced.txt - The classification report dumped into a file by the weight reduced model on the test set.



*The inputs to the program must be in the same folder as the script.

INPUT: The input to the program is cleaneddata_oversampled.csv
OUTPUT:
report:
              precision    recall  f1-score   support

         R&B       0.73      0.73      0.73      5753
     Country       0.86      0.82      0.84      5725
        Jazz       0.95      1.00      0.97      5694
     Hip-Hop       0.86      0.81      0.83      5708
        Rock       0.94      0.99      0.97      5448
        Folk       0.75      0.65      0.69      4340
       Other       0.76      0.76      0.76      5650
       Metal       0.87      0.89      0.88      5433
       Indie       0.54      0.43      0.48      7014
         Pop       0.87      0.96      0.92      5381
    Electronic     0.54      0.61      0.58     10111

    accuracy                           0.77     66257
    macro avg       0.79      0.79      0.79     66257
    weighted avg       0.77      0.77      0.77     66257





Importing the necessary python packages

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk import pos_tag
import nltk
#nltk.download('averaged_perceptron_tagger')
#from langdetect import detect
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
#import emoji
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.externals import joblib 
from sklearn.preprocessing import Normalizer
from scipy.sparse import hstack
from sklearn.metrics import classification_report
from statistics import mean
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from operator import itemgetter 




/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Importing the cleaned oversampled lyrics dataset

In [ ]:
df=pd.read_csv("cleaneddata_oversampled.csv")

(331283, 8)

Dropping the index column whcih was created by when we read a file into a dataframe

In [ ]:
df.drop(["index"], axis = 1, inplace = True)

In [ ]:
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)



Dividing the dataset into train and test set with size of 80% and 20 % respectively.
The train and test datasets are stored into csv files.

In [ ]:
X=df['lyrics']
Y=df['genre']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(Xtrain.shape,Ytrain.shape)
print(Xtest.shape,Ytest.shape)
frame = { 'lyrics': Xtrain, 'genre': Ytrain } 
  
traindf = pd.DataFrame(frame)
traindf.to_csv("oversampling_train.csv")
frame = { 'lyrics': Xtest, 'genre': Ytest } 
  
testdf = pd.DataFrame(frame)
testdf.to_csv("oversampling_test.csv")

(265026,) (265026,)
(66257,) (66257,)


In [ ]:
df.head()

,artist,genre,is_eng,lyrics,song,year
0,atb,Electronic,1.0,till i come till i come chang it and see till ...,9-p-m-till-i-come,2007
1,bobby-darin,Jazz,1.0,if i could talk to the anim just imagin it cha...,talk-to-the-animals,2007
2,erasure,Pop,1.0,love is colder than death they say to warm you...,love-is-colder-than-death,2006
3,alicia-keys,Pop,1.0,let s go back in time when see your id on mine...,love,2007
4,anita-o-day,Other,1.0,yesterday you came my way and when you smile a...,i-can-t-believe-that-you-re-in-love-with-me,2014


In [ ]:
df.head()

,index,artist,genre,is_eng,lyrics,song,year
0,233976,atb,Electronic,1.0,till i come till i come chang it and see till ...,9-p-m-till-i-come,2007
1,132970,bobby-darin,Jazz,1.0,if i could talk to the anim just imagin it cha...,talk-to-the-animals,2007
2,32306,erasure,Pop,1.0,love is colder than death they say to warm you...,love-is-colder-than-death,2006
3,58076,alicia-keys,Pop,1.0,let s go back in time when see your id on mine...,love,2007
4,75486,anita-o-day,Other,1.0,yesterday you came my way and when you smile a...,i-can-t-believe-that-you-re-in-love-with-me,2014


In [ ]:
lyrics=df['lyrics']
genre=df['genre']

Performing cross validation with 5 splits and implementing tfidf vectorization inside the cross validation to avoid any data leakage.



*   TFidf Vectorization will consider parameters stop_words= english specifying that it will be removing any english words and says to consider both unigrams and bigrams.
*   We are specifying the parameters on logistic model as multiclass as multinonial specifying we are seeking a multi class problem and optimization algorithm as 'sag'. Using sag so that the it is best for converging fastly on large datasets.


*   Storing the models in pickle files using joblib  and results with actual and predicted values into a csv file.
*   Storing the accuracy of every cross validation split into a list.





In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=0)
i=0
logirep=[]
logiscore=[]
for train_index, test_index in skf.split(lyrics, genre):
    print(train_index)
    print(test_index)
    x_train1, x_test1 = lyrics.iloc[train_index], lyrics.iloc[test_index]
    y_train, y_test = genre.iloc[train_index], genre.iloc[test_index]
    i=i+1
    tfidf = TfidfVectorizer(stop_words="english",ngram_range=(1,2))
    x_train = tfidf.fit_transform(x_train1)
    x_test = tfidf.transform(x_test1)
    print("tfidf"+str(i))
    
    clf = LogisticRegression(multi_class='multinomial',solver='sag')
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    joblib.dump(clf, 'logiovrsam'+str(i)+'.pkl')
    score = accuracy_score(y_test, y_pred)
    rep=classification_report(y_test, y_pred, target_names=df.genre.unique())
    logirep.append(rep)
    dat={'Actual':y_test,'pred':y_pred}
    resdf=pd.DataFrame(dat)
    resdf.to_csv('reslogiovrsam'+str(i)+'.csv')
    logiscore.append(score)
    print(score)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


[ 64931  64946  64967 ... 331280 331281 331282]
[    0     1     2 ... 68175 68209 68244]
tfidf1
0.7712694507750124
[     0      1      2 ... 331280 331281 331282]
[ 64931  64946  64967 ... 133474 133493 133515]
tfidf2
0.765866248094541
[     0      1      2 ... 331280 331281 331282]
[131532 131542 131549 ... 199969 199978 199982]
tfidf3
0.7694432286399927
[     0      1      2 ... 331280 331281 331282]
[197484 197487 197489 ... 266727 266732 266754]
tfidf4
0.7696510504709008
[     0      1      2 ... 266727 266732 266754]
[263481 263491 263509 ... 331280 331281 331282]
tfidf5
0.7693190050712388


Creating a file to write our classification report for our 5 results of Cross validation models and computing the average accuracy and writing them into the file.

The classification report is computed by using the results csv files containing the actual and predicted values of ech split.

In [ ]:
f=open("logioversamp_results.txt","a")

for i in range(1,6):
    f.write("\n report"+str(i)+":\n")
    dfr=pd.read_csv("reslogiovrsam"+str(i)+".csv")
    dfr.drop(dfr.columns[dfr.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    f.write(classification_report(dfr.Actual, dfr.pred, target_names=df.genre.unique()))
f.write("\nThe average score for this model is")
f.write(str(mean(logiscore)))
f.close()

Declaring the logistic model with multinomial as parameter specifying that we are trying to solve a multi classification problem and sag optimiazation is used as it is best for large datasets and have faster convergence.

In [ ]:
finalmod = LogisticRegression(multi_class='multinomial',solver='sag')

Perform tfidf vectorization to extraxt tfidf matrix on whole training data and use the fitted vectorizer to transform the test data to tfidf matrix.

Tfidf matrix considers:
*   Removing stop words
*   Considering both unigrams and bigrams



In [ ]:
tfidf = TfidfVectorizer(stop_words="english",ngram_range=(1,2))
trainvec = tfidf.fit_transform(lyrics)
testvec = tfidf.transform(Xtest)

Fitting the model.

In [ ]:
finalmod.fit(trainvec, genre)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

Storing the model

In [ ]:
joblib.dump(finalmod, 'Final_models/logiovrsam.pkl')

['/content/drive/My Drive/OverSampling/Final_models/logiovrsam.pkl']

Predicting the test data using test tfidf matrix

In [ ]:
y_pred = finalmod.predict(testvec)

Storing the results into a csv file with actual genre and predicted genre.

In [ ]:
dat={'Actual':Ytest,'pred':y_pred}
resdf=pd.DataFrame(dat)
resdf.to_csv('finalreslogiovrsam.csv')

Computing the classification report and writing the results into a file.

In [ ]:
f=open("finallogioversamp_results.txt","a")
f.write("\n report:\n")
f.write(classification_report(Ytest, y_pred, target_names=Ytest.unique()))
f.close()

In [ ]:
rockweights=finalmod.coef_[10]

In [ ]:
rockweights

array([ 0.02971665,  0.03777732, -0.06487624, ..., -0.01388786,
       -0.00923136, -0.00923136])

In [ ]:
redweights=rockweights*0.5

In [ ]:
redweights

array([ 0.01485832,  0.01888866, -0.03243812, ..., -0.00694393,
       -0.00461568, -0.00461568])

In [ ]:
finalmod.coef_[10]=redweights

In [ ]:
finalmod.coef_[10]

array([ 0.01188612,  0.01511085, -0.02595038, ..., -0.00555515,
       -0.0036925 , -0.0036925 ])

In [ ]:
finalmod.coef_

array([[-0.0329352 , -0.00102394, -0.0083676 , ..., -0.00342399,
        -0.00108113, -0.00108113],
       [ 0.07502783, -0.00866654, -0.05368631, ..., -0.01038293,
        -0.0047271 , -0.0047271 ],
       [-0.02279198, -0.00064929, -0.00612491, ...,  0.06577774,
        -0.00137682, -0.00137682],
       ...,
       [-0.0893871 , -0.00848002, -0.02981261, ..., -0.00890366,
         0.02441166,  0.02441166],
       [ 0.16407221, -0.00300949,  0.23828006, ..., -0.00235363,
        -0.00237978, -0.00237978],
       [ 0.01485832,  0.01888866, -0.03243812, ..., -0.00694393,
        -0.00461568, -0.00461568]])

In [ ]:
y_pred = finalmod.predict(testvec)

In [ ]:
dat={'Actual':Ytest,'pred':y_pred}
resdf=pd.DataFrame(dat)
resdf.to_csv('finalreslogiovrsam_rockreduced.csv')

In [ ]:
f=open("finallogioversamp_results_rockreduced.txt","a")
f.write("\n report:\n")
f.write(classification_report(Ytest, y_pred, target_names=df.genre.unique()))
f.close()

In [ ]:
joblib.dump(clf, 'finallogiovrsam_wholerock'+str(i)+'.pkl')

['finallogiovrsam_wholerock5.pkl']

Selecting only rock records from train dataset

In [ ]:
Rockdf=traindf[traindf['genre']=='Rock']

Resetting the index 

In [ ]:
Rockdf.reset_index(drop=True,inplace=True)

In [ ]:
Rockdf.head()

,lyrics,genre
0,i remeb when i saw you at the diner s yet my h...,Rock
1,i read the paper thi morn i saw your photograp...,Rock
2,well i ve guess your name and i m sure you kno...,Rock
3,shame shame shame ooooh i i i i shame shame sh...,Rock
4,i d realli like to see you now in your father ...,Rock


In [ ]:
rocklyr=Rockdf['lyrics']
rockgenre=Rockdf['genre']

Perform tfidf vectorization to extraxt tfidf matrix on Rock training data and 
Tfidf matrix considers:
*   Removing stop words
*   Considering both unigrams and bigrams



In [ ]:
rtfidf = TfidfVectorizer(stop_words="english",ngram_range=(1,2))
rtrainvec = rtfidf.fit_transform(rocklyr)

Selecting all the records except the rock genre from the training data

In [ ]:
Allnorockdf=traindf[traindf['genre']!='Rock']

In [ ]:
Allnorocklyr=Allnorockdf['lyrics']
Allnorockgenre=Allnorockdf['genre']

Perform tfidf vectorization to extraxt tfidf matrix on all genres except rock from training data and 
Tfidf matrix considers:
*   Removing stop words
*   Considering both unigrams and bigrams



In [ ]:
Allnorocktfidf = TfidfVectorizer(stop_words="english",ngram_range=(1,2))
Allnorockvec = Allnorocktfidf.fit_transform(Allnorocklyr)

Extracting the rock dataset ngrams

In [ ]:
rlist=rtfidf.get_feature_names()

Extracting the ngrams of all genres except the rock

In [ ]:
allnorcklist=Allnorocktfidf.get_feature_names()

Extracting the ngrams of whole training data

In [ ]:
tfidflist=tfidf.get_feature_names()

Finding common ngrams between the rock ngrams and all genres except the rock

In [ ]:
both = set(allnorcklist).intersection(rlist)

In [ ]:
len(allnorcklist)

3205999

In [ ]:
type(allnorcklist)

list

In [ ]:
len(tfidflist)

3731150

In [ ]:
l=list(both)

In [ ]:
len(l)

547094

Finding the common ngrams between the abovengrams(found out from above intersection ) and the ngrams of whole training data.

In [ ]:
intersec2 = set(tfidflist).intersection(l)

In [ ]:
len(intersec2)

547094

In [ ]:
i2=list(intersec2)

In [ ]:
s=np.array(tfidflist)
k=np.array(i2)

Find the indexes of common ngrams(intersection of whole traindata ngrams and the intersection set of the rock and all except rock) from the whole training data .

In [ ]:
Iindex=np.where(np.isin(s,k))

Copying the original rock weights into orrkweights variable.

In [ ]:
orrkweights=finalmod.coef_[10].copy()

In [ ]:
orrkweights

array([-0.0581899 , -0.00016972, -0.06508028, ..., -0.0079137 ,
       -0.01236094, -0.01236094])

This cell is run when we want to get back to the original rock weights in our model.

In [ ]:
finalmod.coef_[10]=orrkweights.copy()

In [ ]:
orrkweights

array([-0.0581899 , -0.00016972, -0.06508028, ..., -0.0079137 ,
       -0.01236094, -0.01236094])

In [ ]:
rkweights=finalmod.coef_[10]

In [ ]:
rkweights

array([-0.0581899 , -0.00016972, -0.06508028, ..., -0.0079137 ,
       -0.01236094, -0.01236094])

In [ ]:
rw=list(rkweights)

Find the rock weights of the common ngrams and reducing by 20%

In [ ]:

list(map(lambda x,y:rkweights.__setitem__(x,rkweights[x]*0.2),list(Iindex[0]),rkweights))
print(rkweights)

[-0.00581899 -0.00016972 -0.06508028 ... -0.0079137  -0.01236094
 -0.01236094]


In [ ]:
np.array(rkweights)

array([-0.00581899, -0.00016972, -0.06508028, ..., -0.0079137 ,
       -0.01236094, -0.01236094])

Initializing the modified weights to our model

In [ ]:
finalmod.coef_[10]=np.array(rkweights)

Predicting the testset with the modified weights

In [ ]:
y_pred = finalmod.predict(testvec)

Storing the results which consist of Actual and predicted values into a csv file.

In [ ]:
dat={'Actual':Ytest,'pred':y_pred}
resdf=pd.DataFrame(dat)
resdf.to_csv('finalreslogiovrsam_rockreduced10.csv')

Writing the classification report into a text file

In [ ]:
f=open("finallogioversamp_intersect_results_rockreduced10.txt","a")
f.write("\n report:\n")
f.write(classification_report(Ytest, y_pred, target_names=Ytest.unique()))
f.close()

Storing the model into a pickle file

In [ ]:
joblib.dump(finalmod, 'finallogiovrsam_reducerock.pkl')

['/content/drive/My Drive/OverSampling/Final_logistic_models/finallogiovrsam_reducerock.pkl']